In [1]:
# 필요한 라이브러리 import
import pymysql
import pandas as pd
import random

# 데이터베이스 연결
db = pymysql.connect(user='hj', password='1234', host='112.161.7.178', db='movie_db', cursorclass=pymysql.cursors.DictCursor)
cursor = db.cursor()

# 배우 테이블 조회
query = "SELECT * FROM 배우"
cursor.execute(query)
actors_data = cursor.fetchall()

# 데이터프레임 생성
db_movie_df = pd.DataFrame(actors_data)

In [2]:
# filmoNames에 데이터를 분리, 랜덤 출력
def separated_string(string):
    choices = string.split("|")
    randomMovieName = random.choice(choices)
    return randomMovieName

In [3]:
# 배우가 한명일 때 실행될 함수
def onePeople(actor, df):

    actor_movies = df[df['peopleNm'] == actor]['filmoNames'].tolist()
    # 검색 배우의 출연작이 없으면 None 리턴
    if not actor_movies:
        return None
    else:
        # 랜덤으로 추천
        recommended_movie = separated_string(random.choice(actor_movies))
        return recommended_movie

In [4]:
# 한명이 아닐 때 실행될 함수
def twoPeople(user_input_actor, df, movie_name):
    
    # 새로운 조건으로 데이터 검색
    query = "SELECT * FROM 배우 WHERE peopleNm = %s AND filmoNames LIKE %s"
    cursor.execute(query, (user_input_actor, f'%{movie_name}%'))
    twoPeopleActorsData = cursor.fetchall()

    # 새로운 데이터프레임 생성
    df = pd.DataFrame(twoPeopleActorsData)
    actor_movies = df[df['peopleNm'] == user_input_actor]['filmoNames'].tolist()
    
    if not actor_movies:
        return None
    else:
        recommended_movie = separated_string(random.choice(actor_movies))
        return recommended_movie

In [5]:
# 검색 결과 출력
def ActorPrint(user_input_actor):
    search_result = db_movie_df[db_movie_df['peopleNm'] == user_input_actor]

    if not search_result.empty:

        # 입력받은 배우가 한명일 때
        if len(search_result) == 1:
            recommended_movie = onePeople(user_input_actor, db_movie_df)
        
            # 결과 출력
            if recommended_movie:
                print("추천 영화:", recommended_movie)
            else:
                print(f"이름이 {user_input_actor}인 배우가 출연한 영화가 없습니다.")

        # 입력받은 배우가 한명이 아닐 때 해당 배우 출연 영화명 입력받기
        else:
            movie_name = input("영화명을 입력하세요: ").strip()
            recommended_movie = twoPeople(user_input_actor, db_movie_df, movie_name)

            # 결과 출력
            if recommended_movie:
                print("추천 영화:", recommended_movie)
            else:
                print(f"이름이 {user_input_actor}인 배우가 출연한 영화가 없습니다.")
    else:
        print(f"입력한 이름 '{user_input_actor}'으로 검색된 배우가 없습니다.")

In [6]:
# 사용자 입력 받기
user_input_actor = ActorPrint(input("배우의 이름을 입력하세요: ").strip())

추천 영화: 좋아해줘


추천 영화: 나의 특별한 형제
